In [76]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model="llama-3.1-8b-instant"
)

response = llm.invoke("The first person to land on moon")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.

Armstrong was followed by Edwin "Buzz" Aldrin, who also walked on the moon during the Apollo 11 mission. Michael Collins remained in orbit around the moon in the command module Columbia, waiting for the lunar module to return to the spacecraft.

Here's a brief timeline of the historic moon landing:

- July 16, 1969: Apollo 11 launches from Kennedy Space Center in Florida.
- July 20, 1969: The lunar module Eagle lands on the moon's surface in the Sea of Tranquility.
- 2:56 UTC (Coordinated Universal Time) on July 21, 1969: Neil Armstrong becomes the first person to walk on the moon.
- July 24, 1969: The Apollo 11 spacecraft returns to Earth, splashing down in the Pacific Oc

In [72]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.jpmorganchase.com/careers")
page_data=loader.load().pop().page_content
print(page_data)




















Careers | JPMorganChase

















Skip to main content






















Join our team
























            About us

            
                
  
  
  
  
  
  
  
  
  












                    About us
                









                    How we do business
                









                    Leadership
                









                    Awards and recognition
                









                    Technology
                









                    Governance
                









                    Suppliers
                









                    Diversity, opportunity & inclusion
                









Read more





Chairman and CEO Letter to Shareholders
Annual Report 2024

Learn more





























            Impact

            
                
  
  
  
  
  
  
  
  
  












                    Impact
                




                   

In [130]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
     """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    This text is from a career/job listing page.
    Extract job postings and return them in JSON format with keys:
    - `role`: Job title
    - `experience`: Years of experience or entry-level/etc.
    - `skills`: Look for phrases like "Qualifications", "Required skills", or "You should have"
    - `description`: Full description of the role

    If a section is not available, try to infer it or use "Not specified".
    ### RETURN VALID JSON ONLY (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input ={'page_data': page_data})

type(res.content)

str

In [131]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

[{'role': 'Lead Associate Operations',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'We see Samuel ...\u2028\u2028 At a company that sees and supports you with the resources you need to\xa0grow your career. Someone passes in front of the camera reveal a bank rep walking through a field of greens. She greets the business owner like an old friend. LOWER THIRD:\xa0Customer relationships\u2028 VO: Joining a global workforce that empowers neighborhoods, nations,\xa0communities and families. SUPER: More than 300k employees in over 65 countries We transition to a community manager as he warmly welcomes members of his community. We cut to him moments later, enthusiastically explaining family finances in front of a whiteboard. LOWER THIRD:\xa0Community impact VO: And that brings together the right people with the resources they need to change the world. SUPER:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 One of Fortune’s “World’s Most Admired Companies” We see community

In [132]:
type(json_res)

list

In [133]:
import pandas as pd

df = pd.read_csv("Techstack_Portfolios.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [134]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [135]:
links = collection.query(
    query_texts=["Experience in Python", "Experience in React"],
    n_results=2
)

links
                         

{'ids': [['650e9109-d98a-4af6-98b7-7dcdbb2aac1b',
   'aced17d4-e8c0-4eba-b3e7-fae470898809'],
  ['fc771d3d-9adf-4be2-9f01-8a2e33363599',
   '7dd910d2-81f4-450d-b09f-415165ee84e4']],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL'],
  ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/react-portfolio'},
   {'links': 'https://example.com/react-native-portfolio'}]],
 'distances': [[0.996884822845459, 1.0578701496124268],
  [0.9630732536315918, 1.0685749053955078]]}

In [150]:
found_skills = False

for job in json_res:
    skills = job.get("skills", "Not specified")
    if skills != "Not specified":
        print(skills)
        found_skills = True

if not found_skills:
    print("No skill data available")



No skill data available


In [160]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

     ### INSTRUCTION:
     You are Vaishnavi Gupta, a SWE intern at Microsoft. Microsoft specializes in AI and Software Engineering.
     Your task is to write a cold email to a potential client regarding the job mentioned above, focusing on how Microsoft can help them achieve their goals and address their specific needs.
     Use the job description to tailor your message, and include relevant achievements or case studies from Microsoft to build credibility.
     Also add the most relevant ones from the following links to showcase Microsoft's portfolio: {link_list}

     Do not include a preamble.

   ### EMAIL (NO PREAMBLE):
   """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock the Power of AI and Software Engineering with Microsoft

Dear [Recipient's Name],

I came across your company and was impressed by your commitment to innovation and excellence. As a SWE intern at Microsoft, I'd like to introduce you to our team and explore how we can help you achieve your goals.

We're currently looking for a Feedback Analyst to join our team, and I believe your organization could greatly benefit from our expertise. Our ideal candidate will have a strong background in [insert relevant skills or experience], which will enable them to drive meaningful insights and improvements in your business.

At Microsoft, we've been at the forefront of AI and software engineering for decades. Our achievements in these areas are a testament to our dedication to innovation and customer satisfaction. For instance, our machine learning capabilities have enabled us to develop solutions that can:

- Predict customer churn with high accuracy, allowing businesses to take proa